#Week 4: Coding Assignment

##Design a prompt to perform the following tasks-

###Customer Review Extraction
1. Extract the customer(name & ID), product and company details
2. Extract the sentiment of the customer
3. What was good or bad about the product- give specific details

#CODE

In [3]:
!pip install pandas nltk gradio -q

In [7]:
import re
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import gradio as gr

nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [8]:
file_path = '/content/customer_review.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    data = file.read()

print(data)

@Customer: Varun D. Kapoor (ID#: CRM5543219) | Purchase Date: 22/10/2023 | Store: Croma Electronics, DLF Mall Branch, Sector 18, Noida, UP - 201301 | Product: Pebble BassX Prime™ Wireless Speaker (#PBL-SPK22) | Order #: CRM/DLF/2023/10/7788 ✨ What a FANTASTIC buy @ just ₹2,499/-! Been using this Pebble speaker for 3 weeks & honestly amazed by the value for money 🎵. Sales rep @Amit (ID#CR567) recommended this over expensive brands & boy was he right! Let me tell you why this budget speaker is punching way above its weight -> First off, the BASS!!! For this price point, totally unexpected depth & clarity = perfect party companion 🔊. Battery life is IMPRESSIVE: advertised 8hrs but I'm getting solid 9.5hrs on 60% volume!!! RGB lights sync w/ music = creates nice ambiance (can be turned off too). Bluetooth 5.0 connection = zero lag & connects instantly w/ my iPhone & laptop. Range is solid, works through walls up to ~30ft ⚡. Delivery/setup experience was smooth: ordered @ 11am, delivered sa

### 1. Details of Customer, Product & Company

In [15]:
customer_match = re.search(r"@Customer: (.+?) \(ID#: (.+?)\)", data)
customer_name = customer_match.group(1) if customer_match else "Not Found"
customer_id = customer_match.group(2) if customer_match else "Not Found"

product_match = re.search(r"Product: (.+?) \((#.+?)\)", data)
product_name = product_match.group(1) if product_match else "Not Found"
product_id = product_match.group(2) if product_match else "Not Found"

store_match = re.search(r"Store: (.+?)\|", data)
store = store_match.group(1) if store_match else "Not Found"


print(f"Customer Name: {customer_name}")
print(f"Customer ID: {customer_id}")
print(f"Product Name: {product_name}")
print(f"Product ID: {product_id}")
print(f"Store: {store}")

Customer Name: Varun D. Kapoor
Customer ID: CRM5543219
Product Name: Pebble BassX Prime™ Wireless Speaker
Product ID: #PBL-SPK22
Store: Croma Electronics, DLF Mall Branch, Sector 18, Noida, UP - 201301 


### 2. Sentimental Analysis of the Customer

In [14]:
sentiment = sia.polarity_scores(data)
sentiment_type = (
    "Positive" if sentiment['compound'] > 0.05 else
    "Negative" if sentiment['compound'] < -0.05 else
    "Neutral"
)

print(f"Sentiment of the customer for the product is '{sentiment_type}'")

Sentiment of the customer for the product is 'Positive'


### 3. Review of the Product

In [22]:
positives = re.findall(r"(?i)\b(?:fantastic|impressive|perfect|great|amazing|helpful|sturdy|smooth|solid)\b", data)
negatives = re.findall(r"(?i)\b(?:tiny|wish|manual)\b", data)


print(f"The good things about the product : {', '.join(set(positives)) if positives else 'Not Found'}")
print(f"The bad things about the product : {', '.join(set(negatives)) if negatives else 'Not Found'}")


The good things about the product : sturdy, helpful, smooth, perfect, solid, great, FANTASTIC, IMPRESSIVE
The bad things about the product : wish, manual, tiny


## Gradio Interface

In [23]:
def analyze_review(review_text):
    # Simplified re-use of code for new reviews
    sentiment = sia.polarity_scores(review_text)
    sentiment_type = (
        "Positive" if sentiment['compound'] > 0.05 else
        "Negative" if sentiment['compound'] < -0.05 else
        "Neutral"
    )

    positives = re.findall(r"(?i)\b(?:fantastic|impressive|perfect|great|amazing|helpful|sturdy|smooth|solid)\b", review_text)
    negatives = re.findall(r"(?i)\b(?:tiny|wish|manual)\b", review_text)

    return {
        "Sentiment": sentiment_type,
        "Positive Feedback": ', '.join(set(positives)) if positives else "Not Found",
        "Negative Feedback": ', '.join(set(negatives)) if negatives else "Not Found"
    }

interface = gr.Interface(
    fn=analyze_review,
    inputs="text",
    outputs="json",
    title="Customer Review Analyzer",
    description="Analyze reviews for sentiment and feedback."
)


interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://16566483a473967c55.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
